In [2]:
import numpy as np
def sum_squares_error(y,t):
    return 0.5*np.sum((y-t)**2)

t = [0,0,1,0,0,0,0,0,0,0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t))

0.09750000000000003

In [3]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t))

0.5975

In [6]:
def cross_entropy_error(y, t): 
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

In [7]:
t = [0,0,1,0,0,0,0,0,0,0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [8]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

2.302584092994546

In [9]:
import sys, os 
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
print(x_train.shape) # (60000, 784) print(t_train.shape) # (60000, 10)

(60000, 784)


In [12]:
train_size = x_train.shape[0]
batch_size = 10

batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask] 
t_batch = t_train[batch_mask]

In [13]:
def numerical_gradient(f, x):
  h = 1e-4 # 0.0001
  grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성
  for idx in range(x.size): 
    tmp_val = x[idx]
    # f(x+h) 계산
    x[idx] = tmp_val + h 
    fxh1 = f(x)
    
    # f(x-h) 계산
    x[idx] = tmp_val - h 
    fxh2 = f(x)
    grad[idx] = (fxh1 - fxh2) / (2*h) 
    x[idx] = tmp_val # 값 복원
  return grad

In [18]:
def gradient_descent(f, init_x, lr, step_num=100):
  x = init_x
  for i in range(step_num):
    grad = numerical_gradient(f, x) 
    x -= lr * grad
  return x

In [19]:
def function_2(x):
  return x[0]**2 + x[1]**2

init_x = np.array([-3.0,4.0])
print(gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100))
print(gradient_descent(function_2, init_x=init_x, lr=10.0, step_num=100))
print(gradient_descent(function_2, init_x=init_x, lr=1e-10, step_num=100))

[-6.11110793e-10  8.14814391e-10]
[ 2.34235971e+12 -3.96091057e+12]
[ 2.34235971e+12 -3.96091057e+12]


In [21]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.functions import softmax, cross_entropy_error 
from common.gradient import numerical_gradient

class simpleNet:
  def __init__(self):
    self.W= np.random.randn(2,3)
  
  def predict(self, x):
    return np.dot(x,self.W)
  
  def loss(self, x, t):
    z = self.predict(x)
    y = softmax(z)
    loss = cross_entropy_error(y,t)

    return loss
    

In [22]:
net = simpleNet()
print(net.W)

[[ 0.14457607  0.66293912 -0.68023818]
 [-0.43779989 -0.10287158 -0.4217258 ]]


In [23]:
x = np.array([0.6,0.9])
p = net.predict(x)
print(p)

[-0.30727426  0.30517906 -0.78769613]


In [24]:
np.argmax(p)

1

In [25]:
t = np.array([0, 1, 0])
net.loss(x,t)

0.6298187738040056

In [29]:
def f(W):
  return net.loss(x, t)
dW = numerical_gradient(f, net.W)
print(dW)

[[ 0.1732438  -0.28039255  0.10714874]
 [ 0.2598657  -0.42058882  0.16072312]]


In [39]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
    # 가중치 초기화
    self.params = {}
    self.params['W1'] = weight_init_std*np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std*np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)

  def predict(self, x):
    W1, W2 = self.params['W1'], self.params['W2']
    b1, b2 = self.params['b1'], self.params['b2']

    a1 = np.dot(x,W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)

    return y
  
  def loss(self, x, t):
    y = self.predict(x)
    return cross_entropy_error(y,t)
  
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    t = np.argmax(t, axis=1)

    accuracy = np.sum(y==1) / float(x.shape[0])
    return accuracy
  
  def numerical_gradient(self, x, t):
    loss_W = lambda W: self.loss(x,t)
    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
    return grads

net = TwoLayerNet(input_size = 784, hidden_size = 100, output_size = 10) 
print(net.params['W1'].shape) # (784, 100)
print(net.params['b1'].shape) # (100,)
print(net.params['W2'].shape) # (100, 10)
print(net.params['b2'].shape) # (10,)


(784, 100)
(100,)
(100, 10)
(10,)


In [40]:
x = np.random.rand(100, 784) # 더미 입력 데이터(100장 분량) 
y = net.predict(x)

In [43]:
import numpy as np
from dataset.mnist import load_mnist 
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list =[]
train_acc_list = []
test_acc_list =[]

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

iter_per_epoch = max(train_size / batch_size, 1)

network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

for i in range(iters_num):
  # 미니배치 흭득
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  # 기울기 계산
  grad = network.numerical_gradient(x_batch, t_batch)

  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]
  
  # 학습 경과 기록
  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)

  if i % iter_per_epoch == 0:
    train_acc = network.accuracy(x_train, t_train) 
    test_acc = network.accuracy(x_test, t_test) 
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print("train acc, test acc | "+ str(train_acc) + ", " + str(test_acc))

train acc, test acc | 0.0, 0.0


KeyboardInterrupt: 

In [ ]:
import numpy as np
from dataset.mnist import load_mnist 
